# Tool recommendation 
## (Gated recurrent units neural network with weighted cross-entropy loss)

In [49]:
import numpy as np
import json
import warnings
import operator

import h5py
from tensorflow.keras.models import model_from_json
from tensorflow.keras import backend as K

warnings.filterwarnings("ignore")


def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data


def create_model(model_path):
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    cu_wt = list()
    for index, item in enumerate(trained_model.keys()):
        if "weight_" in item:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key)[()]
            mean = np.mean(weights)
            cu_wt.append(mean)
            model_weights.append(weights)
            weight_ctr += 1
    print("Overall mean of model weights: %.6f" % np.mean(cu_wt))
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary

def get_predicted_tools(base_tools, predictions, topk):
    """
    Get predicted tools. If predicted tools are less in number, combine them with published tools
    """
    intersection = list(set(predictions).intersection(set(base_tools)))
    return intersection[:topk]

def sort_by_usage(t_list, class_weights,predictions, d_dict):
    """
    Sort predictions by usage/class weights
    """
    tool_dict = dict()
    for tool in t_list:
        t_id = d_dict[tool]
        tool_dict[tool] = predictions[t_id]
    tool_dict = dict(sorted(tool_dict.items(), key=lambda kv: kv[1], reverse=True))
    return list(tool_dict.keys()), list(tool_dict.values())

def separate_predictions(base_tools, prediction, last_tool_name, weight_values, topk):
    """
    Get predictions from published and normal workflows
    """
    last_base_tools = list()
    predictions = prediction * weight_values
    prediction_pos = np.argsort(predictions, axis=-1)
    topk_prediction_pos = prediction_pos[-topk:]
    
    # get tool ids
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    if last_tool_name in base_tools:
        last_base_tools = base_tools[last_tool_name]
        if type(last_base_tools).__name__ == "str":
            # get published or compatible tools for the last tool in a sequence of tools
            last_base_tools = last_base_tools.split(",")
    # get predicted tools
    # p_tools = get_predicted_tools(last_base_tools, pred_tool_ids, topk)
    # sorted_c_t, sorted_c_v = sort_by_usage(p_tools, class_weights,predictions, dictionary)
    sorted_c_v=[predictions[tool_pos] for tool_pos in topk_prediction_pos]
    return pred_tool_ids, sorted_c_v

def compute_recommendations(model, tool_sequence, labels, dictionary, reverse_dictionary, class_weights,model_rec, tool_sequence_rec, labels_rec, dictionary_rec, reverse_dictionary_rec, class_weights_rec, topk=10, max_seq_len=25,toPredict="Filter1"):
    tl_seq = tool_sequence
    tl_seq_ids = [str(dictionary[t]) for t in tl_seq]
    last_tool_name = tl_seq[-1]
    sample = np.zeros(max_seq_len)
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    weight_val_rec = list(class_weights_rec.values())
    weight_val_rec = np.reshape(weight_val_rec, (len(weight_val_rec),))
    for idx, tool_id in enumerate(tl_seq_ids):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    # predict next tools for a test path
    tl_seq_rec = tool_sequence_rec
    tl_seq_ids_rec = [str(dictionary[t]) for t in tl_seq_rec]
    sample_rec = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq_ids_rec):
        sample_rec[idx] = int(tool_id)
    sample_reshaped_rec = np.reshape(sample_rec, (1, max_seq_len))
    predictionLeft = model.predict(sample_reshaped, verbose=0)
    predictionRight = model_rec.predict(sample_reshaped_rec, verbose=0)
    
    nw_dimension = predictionLeft.shape[1]
    predictionLeft = np.reshape(predictionLeft, (nw_dimension,))
    
    nw_dimension_rec = predictionRight.shape[1]
    predictionRight = np.reshape(predictionRight, (nw_dimension,))
    half_len = int(nw_dimension / 2)
    half_len_rec = int(nw_dimension_rec / 2)
    
    pub_t, pub_v = separate_predictions(standard_connections, predictionLeft[:half_len], last_tool_name, weight_val, topk)
    pub_t_rec, pub_v_rec = separate_predictions(standard_connections_rec, predictionRight[:half_len_rec], last_tool_name, weight_val_rec, topk)
    

    # get recommended tools from normal workflows
    c_t, c_v = separate_predictions(compatible_tools, predictionLeft[half_len:], last_tool_name, weight_val, topk)
    c_t_rec, c_v_rec = separate_predictions(compatible_tools_rec, predictionRight[half_len_rec:], last_tool_name, weight_val_rec, topk)
    
    # combine predictions coming from different workflows
    # promote recommended tools coming from published workflows
    # to the top and then show other recommendations
    pub_t.extend(c_t)
    pub_v.extend(c_v)
    pub_t_rec.extend(c_t_rec)
    pub_v_rec.extend(c_v_rec)
    # remove duplicates if any
    pub_t = list(dict.fromkeys(pub_t))
    pub_v = list(dict.fromkeys(pub_v))
    pub_t_rec = list(dict.fromkeys(pub_t_rec))
    pub_v_rec = list(dict.fromkeys(pub_v_rec))
    intop=0
    if (toPredict in pub_t) or (last_tool_name in pub_t_rec):
        intop=1
        if (not (last_tool_name in pub_t_rec)):
            weight_score=pub_v[pub_t.index(toPredict)]
        else:
            if (not (toPredict in pub_t)):
                weight_score=pub_v_rec[pub_t_rec.index(last_tool_name)]
            else:
                weight_score=1-(1-pub_v[pub_t.index(toPredict)])*(1-pub_v_rec[pub_t_rec.index(last_tool_name)])
    rank=0
    if intop==1:
        rank=21
        for i in pub_v:
            for j in pub_v_rec:
                if (1-(1-i)*(1-j))>weight_score:
                    rank-=1
    return rank,intop



## Unpack trained model for prediction

In [50]:
model_path = "data/tool_recommendation_model_20_05.hdf5"
trained_model = h5py.File(model_path, 'r')
model_config = json.loads(trained_model.get('model_config')[()])
dictionary = json.loads(trained_model.get('data_dictionary')[()])
class_weights = json.loads(trained_model.get('class_weights')[()])
standard_connections = json.loads(trained_model.get('standard_connections')[()])
compatible_tools = json.loads(trained_model.get('compatible_tools')[()])
loaded_model = model_from_json(model_config)
model, dictionary, reverse_dictionary = create_model(model_path)
model_path_rec = "data/tool_recommendation_model_rec.hdf5"
trained_model_rec = h5py.File(model_path_rec, 'r')
model_config_rec = json.loads(trained_model_rec.get('model_config')[()])
dictionary_rec = json.loads(trained_model_rec.get('data_dictionary')[()])
class_weights_rec = json.loads(trained_model_rec.get('class_weights')[()])
standard_connections_rec = json.loads(trained_model_rec.get('standard_connections')[()])
compatible_tools_rec = json.loads(trained_model_rec.get('compatible_tools')[()])
loaded_model_rec = model_from_json(model_config_rec)
model_rec, dictionary_rec, reverse_dictionary_rec = create_model(model_path_rec)

Overall mean of model weights: -0.075522
Overall mean of model weights: -0.075522


## Example tools

In [51]:
# Assembly: 
# (https://training.galaxyproject.org/training-material/topics/assembly/tutorials/debruijn-graph-assembly/tutorial.html)
# spades -> 'bandage_info', 'fasta-stats', 'bandage_image', 'fasta_filter_by_length', 'abricate', 'quast', 'mlst' ... 
# velveth -> velvetg
# (https://training.galaxyproject.org/training-material/topics/assembly/tutorials/unicycler-assembly/tutorial.html)
# unicycler -> 'bandage_info', 'glimmer_build-icm', 'glimmer_knowlegde-based', 'bandage_image', 'transdecoder', 'minimap2', 'antismash', 'fasta_filter_by_length' ...


## Computational chemistry
# ctb_remDuplicates -> ctb_remIons 
# ctb_remDuplicates,ctb_remIons -> 'ctb_chemfp_mol2fps', 'ctb_compound_convert'
# ctb_remDuplicates,ctb_remIons,ctb_chemfp_mol2fps -> 'ctb_chemfp_butina_clustering', 'ctb_simsearch', 'ctb_chemfp_nxn_clustering', 'comp1'
# (https://training.galaxyproject.org/training-material/topics/computational-chemistry/tutorials/cheminformatics/tutorial.html)


## RAD-seq
# (https://training.galaxyproject.org/training-material/topics/ecology/tutorials/ref-based-rad-seq/tutorial.html)
# stacks_procrad -> 'bwa', 'bwa_wrapper', 'Grep1', 'stacks_denovomap', 'fastqc', 'fastq_filter'


## Epigenetics
# (https://training.galaxyproject.org/training-material/topics/epigenetics/tutorials/atac-seq/tutorial.html)
# cutadapt,bowtie2 => samtools_flagstat', 'picard_MarkDuplicates', 'picard_AddOrReplaceReadGroups', 'macs2_callpeak','bg_sortmerna', 'multiqc', 'hisat2', 'trim_galore', 'bowtie2' ...
# cutadapt,bowtie2,picard_MarkDuplicates -> 'picard_ReorderSam', 'gatk4_mutect2', 'samtools_rmdup'
# cutadapt,bowtie2,picard_MarkDuplicates,genrich -> 'pygenomeTracks'
#
# (https://training.galaxyproject.org/training-material/topics/epigenetics/tutorials/methylation-seq/tutorial.html)
# bwameth -> 'samtools_rmdup', 'samtools_sort', 'bam_to_sam', 'pileometh' ..
# bwameth,pileometh -> 'deeptools_compute_matrix', 'tp_sed_tool', 'Filter1', 'metilene', 'Remove beginning1', 'wig_to_bigWig'
#
# bowtie2 -> samtools_flagstat', 'picard_MarkDuplicates', 'picard_AddOrReplaceReadGroups ... 
# bowtie2,deeptools_multi_bam_summary -> 'deeptools_plot_pca', 'r_correlation_matrix' ...
# (https://training.galaxyproject.org/training-material/topics/epigenetics/tutorials/formation_of_super-structures_on_xi/tutorial.html)

# bowtie2,hicexplorer_hicbuildmatrix -> 'hicexplorer_hicsummatrices', 'hicexplorer_hicplotviewpoint', 'tp_sed_tool', 'hicexplorer_hiccorrectmatrix', 'hicexplorer_hicmergematrixbins', 'hicexplorer_hicpca' ..
# bowtie2,hicexplorer_hicbuildmatrix,hicexplorer_hicmergematrixbins -> 'hicexplorer_hiccorrectmatrix', 'hicexplorer_hicplottads', 'hicexplorer_hicplotmatrix'
# (https://training.galaxyproject.org/training-material/topics/epigenetics/tutorials/hicexplorer/tutorial.html)

# minfi_read450k -> 'minfi_getbeta'


## Genome annotation
# (https://training.galaxyproject.org/training-material/topics/genome-annotation/tutorials/annotation-with-maker/tutorial.html)
# maker -> 'gffread', 'maker_map_ids', 'jcvi_gff_stats'
# (https://training.galaxyproject.org/training-material/topics/genome-annotation/tutorials/annotation-with-prokka/tutorial.html)
# prokka -> 'mlst', 'jbrowse', 'taxonomy_krona_chart' ...


## Imaging
# (https://training.galaxyproject.org/training-material/topics/imaging/tutorials/hela-screen-analysis/tutorial.html)
# ip_filter_standard -> 'ip_histogram_equalization', 'ip_threshold', 'ip_count_objects
# ip_filter_standard,ip_threshold -> ip_binary_to_labelimage', 'ip_2d_split_binaryimage_by_watershed', 'ip_count_objects', 'ip_convertimage'
# ip_filter_standard,ip_threshold,ip_2d_split_binaryimage_by_watershed -> 'ip_2d_filter_segmentation_by_features', 'ip_2d_feature_extraction'


## Mass spectrometry
# mass_spectrometry_imaging_preprocessing -> 'mass_spectrometry_imaging_combine', 'mass_spectrometry_imaging_preprocessing'
# mass_spectrometry_imaging_preprocessing,mass_spectrometry_imaging_combine -> 'maldi_quant_preprocessing', 'mass_spectrometry_imaging_preprocessing', 'mass_spectrometry_imaging_qc' ...
# search_gui -> peptide_shaker
# search_gui,peptide_shaker -> 'mz_to_sqlite', 'Remove beginning1', 'tp_replace_in_column', 'unipept', proteomics_moff ...


## Single cell
# raceid_main -> 'seurat','raceid_trajectory'
# raceid_main,raceid_trajectory -> 'raceid_inspecttrajectory'
# raceid_inspectclusters,__BUILD_LIST__ -> 'picard_MarkDuplicates', 'hisat2', 'stringtie_merge', 'cutadapt', 'tp_cat'
# raceid_filtnormconf -> 'raceid_clustering', '__BUILD_LIST__'
# raceid_filtnormconf,raceid_clustering -> 'raceid_trajectory', 'raceid_inspectclusters'
# scanpy_regress_variable -> scanpy_scale_data 
# scanpy_regress_variable,scanpy_scale_data -> 'scanpy_run_pca', 'scanpy_find_variable_genes'
# scanpy_regress_variable,scanpy_scale_data,scanpy_run_pca -> 'scanpy_compute_graph', 'scanpy_plot', 'scanpy_run_tsne', 'scanpy_plot_embed'

# (https://training.galaxyproject.org/training-material/topics/transcriptomics/tutorials/scrna-preprocessing-tenx/tutorial.html)
# rna_starsolo -> 'dropletutils', 'multiqc'
# rna_starsolo,dropletutils -> 'scanpy_read_10x', 'scanpy_cluster_reduce_dimension', 'seurat_read10x', 'anndata_import', 'scanpy_plot', 'raceid_filtnormconf'


## Variant calling
# (https://training.galaxyproject.org/training-material/topics/variant-analysis/tutorials/microbial-variants/tutorial.html)
# snippy -> 'bedtools_intersectbed', 'vcfvcfintersect', 'Remove beginning1', 'snippy_core', 'qualimap_bamqc', 'freebayes', 'jbrowse', 'vcfcombine'

# https://training.galaxyproject.org/training-material/topics/variant-analysis/tutorials/somatic-variants/tutorial.html
# trimmomatic,bwa_mem,samtools_rmdup,bamleftalign -> 'samtool_filter2', 'ivar_variants', 'freebayes', 'varscan_somatic', 'deeptools_bam_coverage', 'fastqc', 'ngsutils_bam_filter', 'bamFilter', 'rgPicFixMate', 'samtools_calmd'
# trimmomatic,bwa_mem,samtools_rmdup,bamleftalign,varscan_somatic -> 'bcftools_norm', 'gemini_annotate', 'vt_normalize', 'vcffilter2', 'vcfallelicprimitives', 'snpEff'

# (https://training.galaxyproject.org/training-material/topics/variant-analysis/tutorials/dip/tutorial.html)
# freebayes -> 'vcfallelicprimitives', 'bcftools_norm', 'custom_pro_db', 'vcfvcfintersect'
# freebayes,vcfallelicprimitives -> 'vt_normalize', 'snpSift_filter', 'snpSift_annotate', 'snpEff'
# freebayes,vcfallelicprimitives,snpEff -> 'gemini_load', 'mimodd_varreport', 'snpSift_extractFields


# Transcriptomics
#(https://training.galaxyproject.org/training-material/topics/transcriptomics/tutorials/small_ncrna_clustering/tutorial.html)
# samtools_sort,blockclust,sort1 -> 'cshl_awk_tool', 'Show beginning1', 'tp_awk_tool', 'blockbuster'

# (https://training.galaxyproject.org/training-material/topics/transcriptomics/tutorials/ref-based/tutorial.html)
# cutadapt -> 'umi_tools_extract', 'fastq_paired_end_interlacer', 'chira_collapse', 'rna_star',
# cutadapt,rna_star -> 'featurecounts', 'multiqc', 'htseq_count', 'rseqc_infer_experiment', 'samtools_stats', 'bamFilter',
# cutadapt,rna_star,featurecounts -> 'multiqc', 'deseq2', 'tp_sort_header_tool', 'bamFilter', 'collection_column_join ...


# Single-cell HiC
# schicexplorer_schicqualitycontrol -> 'schicexplorer_schicnormalize'
# schicexplorer_schicnormalize -> 'schicexplorer_schicclustersvl', 'schicexplorer_schicconsensusmatrices', 'schicexplorer_schicplotclusterprofiles'
# schicexplorer_schicnormalize,schicexplorer_schicclustersvl -> 'schicexplorer_schicplotclusterprofiles', 'schicexplorer_schicconsensusmatrices'


# Animal detection on acoustic recording
# vigiechiro_idvalid -> 'vigiechiro_bilanenrichipf', 'vigiechiro_bilanenrichirp'

## Recommended tools

In [52]:
import random
topk = 10 # set the maximum number of recommendations
# Give tools ids in a sequence and see the recommendations. # To know all the tool ids, 
# please print the variable 'reverse_dictionary'
def singTest(testRund):
    total_prediction_score=0
    total_weight_score=0
    total_rank=0
    total_prediction_score_org=0
    total_weight_score_org=0
    insequence=0
    numTest=testRund
    numRealTest=0
    ranks=[]
    with open('data/unique_paths.txt', 'r') as file:
        data = file.read()[1:-1].split(", ")
        print(len(data))
        for index in range(numTest):
            k=random.randrange(len(data))
            i=data[k]
            j=i[1:-1].split(",")
            if (len(j)-1>1 and len(j)<25):
                k=random.randrange(1,len(j)-1,1)
                testPath=j[:k]
                testPath_rec=j[k+1:]
                rank,intop=compute_recommendations(model, testPath, "", dictionary, reverse_dictionary, class_weights,model_rec, testPath_rec, "", dictionary_rec, reverse_dictionary_rec, class_weights_rec, topk,toPredict=j[k])
                if intop==1:
                    total_rank+=rank
                insequence+=intop
                numRealTest+=1
            if (insequence>total_rank):
                print(insequence/numRealTest,' ',total_rank/insequence)
            
        return insequence/numRealTest,total_rank/insequence

rank_list=[]
acc_list=[]
for i in range(10):
    acc,rank=singTest(1000)
    rank_list.append(rank)
    acc_list.append(acc)

with open(r'data/accInBiDirect.txt', 'w') as fp:
    json.dump(acc_list, fp)

rankFileName='data/rankInBiDirect.txt'
with open(rankFileName, 'w') as fp:
    json.dump(rank_list, fp)


198910
1.0   -33.0
1.0   -25.0
1.0   -148.0
1.0   -285.75
1.0   -227.0
1.0   -192.0
1.0   -164.14285714285714
1.0   -315.25
1.0   -349.0
1.0   -313.3
1.0   -285.09090909090907
1.0   -377.0833333333333
1.0   -348.46153846153845
1.0   -324.64285714285717
1.0   -335.6666666666667
1.0   -317.875
1.0   -306.4117647058824
1.0   -288.94444444444446
1.0   -272.63157894736844
1.0   -259.75
0.9523809523809523   -259.75
0.9090909090909091   -259.75
0.9130434782608695   -272.6666666666667
0.9166666666666666   -321.1363636363636
0.92   -309.0
0.9230769230769231   -295.25
0.9259259259259259   -285.48
0.9285714285714286   -284.46153846153845
0.9310344827586207   -275.14814814814815
0.9   -275.14814814814815
0.9032258064516129   -274.57142857142856
0.90625   -286.44827586206895
0.9090909090909091   -285.53333333333336
0.8823529411764706   -285.53333333333336
0.8857142857142857   -282.03225806451616
0.8888888888888888   -273.3125
0.8918918918918919   -264.93939393939394
0.8918918918918919   -264.939393

KeyboardInterrupt: 